In [1]:
 %%writefile all.cpp
#include <iostream>
#include <vector>
#include <omp.h>
#include <climits>
#include <chrono>

using namespace std;

void min_reduction(vector<int>& arr) {
    int min_value = INT_MAX;
    #pragma omp parallel for reduction(min: min_value)
    for (int i = 0; i < arr.size(); i++) {
        if (arr[i] < min_value) {
            min_value = arr[i];
        }
    }
    cout << "Minimum value: " << min_value << endl;
}

void max_reduction(vector<int>& arr) {
    int max_value = INT_MIN;
    #pragma omp parallel for reduction(max: max_value)
    for (int i = 0; i < arr.size(); i++) {
        if (arr[i] > max_value) {
            max_value = arr[i];
        }
    }
    cout << "Maximum value: " << max_value << endl;
}

void sum_reduction(vector<int>& arr) {
    int sum = 0;
    #pragma omp parallel for reduction(+: sum)
    for (int i = 0; i < arr.size(); i++) {
        sum += arr[i];
    }
    cout << "Sum: " << sum << endl;
}

void average_reduction(vector<int>& arr) {
    int sum = 0;
    #pragma omp parallel for reduction(+: sum)
    for (int i = 0; i < arr.size(); i++) {
        sum += arr[i];
    }
    cout << "Average: " << (double)sum / arr.size() << endl;
}

int main() {
    int n;
    cout << "Enter the number of elements: ";
    cin >> n;

    vector<int> arr(n);
    cout << "Enter " << n << " elements:\n";
    for (int i = 0; i < n; ++i) {
        cin >> arr[i];
    }

    auto start_time = chrono::high_resolution_clock::now();

    min_reduction(arr);
    max_reduction(arr);
    sum_reduction(arr);
    average_reduction(arr);

    auto end_time = chrono::high_resolution_clock::now();
    auto duration = chrono::duration_cast<chrono::microseconds>(end_time - start_time).count();

    cout << "Time taken: " << duration << " microseconds" << endl;
    cout << "Number of threads: " << omp_get_max_threads() << endl;

    return 0;
}


Writing all.cpp


In [2]:
!g++ -fopenmp all.cpp -o Myexe

In [3]:
!./Myexe

Enter the number of elements: 5
Enter 5 elements:
1 
2
3
4
5
Minimum value: 1
Maximum value: 5
Sum: 15
Average: 3
Time taken: 211 microseconds
Number of threads: 2


In [4]:

# 1. **Time Complexity**:
#    - Question: What is the time complexity of your parallel reduction operations?
#    - Answer: The time complexity of each parallel reduction operation (min, max, sum, average) is O(log n), where n is the number of elements in the array. This is because the array is divided among threads, each performing a portion of the reduction, and then the partial results are combined logarithmically until the final result is obtained.

# 2. **Comparing Sequential vs. Parallel**:
#    - Question: How does the parallel implementation improve performance compared to a sequential implementation?
#    - Answer: In a sequential implementation, these operations have a time complexity of O(n), where n is the number of elements in the array, because each operation requires iterating through the entire array. However, in the parallel implementation, the workload is distributed among multiple threads, allowing multiple elements to be processed simultaneously. This reduces the overall execution time, especially for large arrays, making the parallel implementation more efficient.

# 3. **Thread Management**:
#    - Question: How does OpenMP manage threads in your code?
#    - Answer: OpenMP manages threads automatically by creating a team of threads at the beginning of the parallel region and distributing the work among them. The number of threads can be controlled using environment variables or runtime functions. In our code, we use `omp_get_max_threads()` to retrieve the maximum number of threads available for parallel execution.

# 4. **Load Balancing**:
#    - Question: How does your code ensure load balancing among threads?
#    - Answer: OpenMP automatically distributes the workload evenly among threads in the parallel region. Each thread processes a contiguous subset of the array, ensuring that the work is balanced. Additionally, the reduction operation efficiently combines the partial results produced by each thread.

# 5. **Overhead**:
#    - Question: Does parallelization introduce any overhead? If so, how is it managed?
#    - Answer: Yes, parallelization introduces overhead due to thread creation, synchronization, and combining partial results. However, the benefits of parallel execution often outweigh this overhead, especially for large arrays. OpenMP manages overhead by optimizing thread creation and synchronization mechanisms.

# 6. **Thread Synchronization**:
#    - Question: How does your code ensure proper synchronization among threads?
#    - Answer: Synchronization is achieved implicitly through OpenMP's reduction clause. Each thread maintains its own copy of the reduction variable (`min_value`, `max_value`, `sum`), and at the end of the parallel region, the partial results are combined using the specified reduction operation (min, max, +). This ensures thread safety without the need for explicit synchronization primitives.

# These questions and answers provide insights into the performance, scalability, and thread management aspects of your parallel reduction operations.